In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
bd = pd.read_csv('Tech_Eval/blockchain_data.csv')
md = pd.read_csv('Tech_Eval/market_data.csv')

## Setting up a bitcoin value we care about. 

inter = bd[['asset_id', 'block_reward_sum']]
bd_btc = inter[inter['asset_id'] == 'Bitcoin_BTC_BTC']
bd_btc = bd_btc['block_reward_sum']

## Basic Analysis

What we're interested in here is the absolute most basic kinds of metrics we might want to see: moving averages, percentage change, that kind of thing. 

### Rolling Mean 

In [ ]:
btc_rolling = bd_btc.rolling(10).mean().reset_index().drop('index', axis=1)
print(btc_rolling[10:30])

In [ ]:
btc_rolling.plot()
plt.show()

#### Why do I care about rolling means? 

#### Future developments in rolling mean.

### Percent Change 

In [ ]:
bd_pct = bd_btc[:20].pct_change(19)
bd_pct[-1:]

#### Why do I care about Percent Change? 

#### Future developments in Percent Change. 

## Fundamental Analysis 

#### NVT (Network Value to Transaction Ratio)
[Source](http://charts.woobull.com/bitcoin-nvt-ratio/)

In [ ]:
# NVT = market cap / total daily transactions.
# Network value = market capitalization
# market cap = total number of assets * price per asset.
# total number of assets = bitcoin_data['supply_total'] 
# price per asset = market_data['price_close']. Choosing 'price_close' is arbitrary. 

# This is essentially the crypto analogue to P/E ratios. 

num_assets = bd.loc[0, ['supply_total']][0] # The total number of btc assets on 2-19-2019
price_per_asset = md.loc[0, ['price_close']][0] # Closing price of btc asset on 2-19-2019
trx_vol = bd.loc[0, ['txn_count']][0]

market_cap = (num_assets * price_per_asset)
NVT = (num_assets * price_per_asset)/trx_vol
print(NVT)

#### Why do I care about NVT?

#### Future developments in NVT.

### MVRV (Market Cap to Realized Capital Ratio)
[Source](http://charts.woobull.com/bitcoin-mvrv-ratio/)

In [ ]:
# MVRV = market cap / realized value
# realized value = total volume of assets * price at last movement on the blockchain. 
# Market cap is defined above.
# I think total volume can be taken as before from bd['supply_total'], though maybe I'm misinterpreting that. 
# None of the columns look like they contain 'price at last movement on the blockchain', so there's no way to do that.

# Tends to be used in discovering market peaks and discerning when a coin is underpriced.

# MVRV = (market_cap)/(num_assets * price at last mvmt on blockchain, a value which I can't currently calculate)

#### Why do I care about MVRV?

#### Future developments in MVRV. 

## Technical Analysis 